# Network Dissection (for classifiers)

In this notebook, we will examine internal layer representations for a classifier trained to recognize scene categories.

Setup matplotlib, torch, and numpy for a high-resolution browser.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib as mpl
from importlib import reload
import IPython
mpl.rcParams['lines.linewidth'] = 0.25
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.linewidth'] = 0.25

Set up experiment directory and settings

In [ ]:
import torch, argparse, os, shutil, inspect, json, numpy, math
import netdissect
from netdissect.easydict import EasyDict
from netdissect import pbar, nethook, renormalize, parallelfolder, pidfile
from netdissect import upsample, tally, imgviz, imgsave, bargraph, show
from experiment import dissect_experiment

model_name='DA-SS-sketch'
# choices are alexnet, vgg16, or resnet152.
args = EasyDict(model=model_name, dataset='pacs-c', seg='netpqc', layer='layer0', quantile=0.01)
resdir = 'results/%s-%s-%s-%s-%s' % (args.model, args.dataset, args.seg, args.layer, int(args.quantile * 1000))
print(resdir)
def resfile(f):
    return os.path.join(resdir, f)


torch.cuda.set_device(6)

model = torch.load('./'+model_name+'-t001-torch16-0.pkl',map_location='cpu')
model = model.cuda()

In [ ]:
dataset = dissect_experiment.load_dataset(args)
sample_size = len(dataset)

print(len(dataset))
print(dataset[0][0].shape)
print(dataset[2][1])

# Classifier labels
from urllib.request import urlopen
from netdissect import renormalize

classlabels = dataset.classes
print(classlabels)
renorm = renormalize.renormalizer(dataset, target='zc')

# 由于 base 环境配置问题，暂时无法加载 Segmentation 模型
# segmodel, seglabels, segcatlabels = experiment.setting.load_segmenter(args.seg)

load classifier model and dataset

In [ ]:
# torch.cuda.set_device(6)

# model = experiment.load_model(args)
# layername = experiment.instrumented_layername(args)
# model.retain_layer(layername)
# dataset = experiment.load_dataset(args)
# upfn = experiment.make_upfn(args, dataset, model, layername)
# sample_size = len(dataset)
# percent_level = 1.0 - args.quantile

# print('Inspecting layer %s of model %s on %s' % (layername, args.model, args.dataset))

Load segmenter, segment labels, classifier labels

Test classifier on some images

In [ ]:
from netdissect import renormalize

indices = [624]
layername = 'layer2'

# None,... 的用法是改变数据的纬度，在指定位置增加一维
# batch.shape: B*Channel*H*W

batch = torch.cat([dataset[i][0][None,...] for i in indices])
truth = [classlabels[dataset[i][1]] for i in indices]
#res.shape=B*C => 8 * 365(No. of classes)
activations, res = model(batch.cuda())
preds = res.max(1)[1]
imgs = [renormalize.as_image(t, source=dataset) for t in batch]
prednames = [classlabels[p.item()] for p in preds]

show([[img, 'pred: ' + pred, 'true: ' + gt] for img, pred, gt in zip(imgs, prednames, truth)])

from netdissect import imgviz
acts = activations[layername].detach().cpu()

test = acts[0][18].view(1, -1)
print((test == 0).sum(), test.shape)
print(acts[0][18].sum())

ivsmall = imgviz.ImageVisualizer((100, 100), source=dataset)
# u 是通道编号，对于 conv5_3 来说就是 min(512, 12)
display(show.blocks(
    [[[ivsmall.masked_image(batch[0], acts, (0, u), percent_level=0.10)],
      [ivsmall.heatmap(acts, (0, u), mode='nearest')]] for u in [6, 20, 73, 91]]
))

num_units = acts.shape[1]

segment single image, and visualize the labels

In [ ]:
# from netdissect import imgviz

# iv = imgviz.ImageVisualizer(120, source=dataset)
# seg = segmodel.segment_batch(renorm(batch).cuda(), downsample=4)

# show([(iv.image(batch[i]), iv.segmentation(seg[i,0]),
#             iv.segment_key(seg[i,-1], segmodel))
#             for i in range(len(seg))])
print(activations.keys())
print(activations['max_pool'].sum())

upfn = dissect_experiment.make_upfn_without_hooks(args, dataset, layername, activations)
percent_level = 1.0 - args.quantile

visualize activations for single layer of single image

## Collect quantile statistics

First, unconditional quantiles over the activations.  We will upsample them to 56x56 to match with segmentations later.

In [ ]:
pbar.descnext('rq')
def compute_samples(batch, *args):
    image_batch = batch.cuda()
    activations, _ = model(image_batch)
    acts = activations[layername].detach()
    hacts = upfn(acts)
    return hacts.permute(0, 2, 3, 1).contiguous().view(-1, acts.shape[1])
rq = tally.tally_quantile(compute_samples, dataset,
                          sample_size=sample_size,
                          r=8192,
                          num_workers=100,
                          pin_memory=True,
                          cachefile=resfile('rq.npz'))


pbar.descnext('topk')
def compute_image_max(batch, *args):
    image_batch = batch.cuda()
    activations, _ = model(image_batch)
    acts = activations[layername].detach()
    acts = acts.view(acts.shape[0], acts.shape[1], -1)
    acts = acts.max(2)[0]
    return acts

#topk => (64X100, 64X100)
topk = tally.tally_topk(compute_image_max, dataset, sample_size=sample_size,
        batch_size=50, num_workers=30, pin_memory=True,
        cachefile=resfile('topk.npz'))

In [ ]:
# single image visualization
rank=0
percent_level=0.10
unit_number = 73
layername='layer2'

print(topk.result()[0][unit_number][rank].item(), 
      topk.result()[1][unit_number][rank].item(), 
      dataset.images[topk.result()[1][unit_number][rank]])

image_number = topk.result()[1][unit_number][rank].item()
# image_number=624
# unit_number=18

iv = imgviz.ImageVisualizer((224, 224), source=dataset, quantiles=rq,
        level=rq.quantiles(percent_level))
batch = torch.cat([dataset[i][0][None,...] for i in [image_number]])
truth = [classlabels[dataset[i][1]] for i in [image_number]]

activations, output = model(batch.cuda())

preds = output.max(1)[1]
acts = activations[layername].detach()

imgs = [renormalize.as_image(t, source=dataset) for t in batch]
prednames = [classlabels[p.item()] for p in preds]

show([[img, 'pred: ' + pred, 'true: ' + gt] for img, pred, gt in zip(imgs, prednames, truth)])
show([[iv.masked_image(batch[0], acts, (0, unit_number))]])


In [ ]:
print(topk.result()[0].shape)
print(topk.result()[1].shape)
print(topk.result()[0][18][0], topk.result()[1][18][0])
print(topk.result()[0][18][1], topk.result()[1][18][1])
print(topk.result()[0][18][2], topk.result()[1][18][2])
print(topk.result()[0][10][99], topk.result()[1][10][99])

Then we just need to run through and visualize the images.

In [ ]:
pbar.descnext('unit_images')

iv = imgviz.ImageVisualizer((100, 100), source=dataset, quantiles=rq,
        level=rq.quantiles(percent_level))
def compute_acts(*image_batch):
    image_batch = image_batch[0].cuda()
    activations, _ = model(image_batch)
    acts_batch = activations[layername]
    return acts_batch
unit_images = iv.masked_images_for_topk(
        compute_acts, dataset, topk, k=5, num_workers=30, pin_memory=True,
        cachefile=resfile('top5images.npz'))

In [ ]:
for u in [17, 18, 36, 38, 48]:
    print( 'unit %d' % u)
    display(unit_images[u])

## Label Units

Collect 99% quantile stats.

In [ ]:
level_at_99 = rq.quantiles(percent_level).cuda()[None,:,None,None]
# Use the segmodel for segmentations.  With broden, we could use ground truth instead.
def compute_conditional_indicator(batch, *args):
    image_batch = batch.cuda()
    seg = segmodel.segment_batch(renorm(image_batch), downsample=4)
    _ = model(image_batch)
    acts = model.retained_layer(layername)
    hacts = upfn(acts)
    iacts = (hacts > level_at_99).float() # indicator
    return tally.conditional_samples(iacts, seg)
pbar.descnext('condi99')
condi99 = tally.tally_conditional_mean(compute_conditional_indicator,
        dataset, sample_size=sample_size,
        num_workers=3, pin_memory=True,
        cachefile=resfile('condi99.npz'))

In [ ]:
iou_99 = tally.iou_from_conditional_indicator_mean(condi99)
unit_label_99 = [
        (concept.item(), seglabels[concept], segcatlabels[concept], bestiou.item())
        for (bestiou, concept) in zip(*iou_99.max(0))]
label_list = [labelcat for concept, label, labelcat, iou in unit_label_99 if iou > 0.04]
display(IPython.display.SVG(experiment.graph_conceptcatlist(label_list)))
len(label_list)

Show a few units with their labels

In [ ]:
for u in [10, 20, 30, 40]:
    print('unit %d, label %s, iou %.3f' % (u, unit_label_99[u][1], unit_label_99[u][3]))
    display(unit_images[u])